## Recipe Analysis

### Load recipe and pre-load nutrition information

In [1]:
from nourishTools import load_json
import pandas as pd
import sys

# Load recipe
recipe_file = r"C:\Users\Fer\Desktop\Code\nourish\data\recipes\ramen.json"
recipe_data = load_json(recipe_file)

# Load product info
prod_data = pd.read_csv(r"C:\Users\Fer\Desktop\Code\nourish\data\productdb.csv")
# display(prod_data)


### Recipe metadata

In [2]:
recipe_name = recipe_data["name"]
recipe_servings = recipe_data["servings"]

print("Recipe: {}, Servings: {}".format(recipe_name, recipe_servings))

Recipe: Ramen, Servings: 2


### Create recipe dataframe

In [3]:
# Create recipe data frame
nutrients = [
    'Calories',
    'Protein',
    'Fiber',
    'TotalCarboHydrates',
    'TotalFat',
    'SaturatedFat',
    'TransFat',
    'PolyUnsaturatedFat',
    'MonoUnsaturatedFat',
    'AddedSugars',
    'Cholesterol',
    'Calcium',
    'Iron',
    'Magnesium',
    'Potassium',
    'Sodium',
    'Zinc',
    'Vit A',
    'Vit B6',
    'Vit B12',
    'Vit C',
    'Vit D',
    'Vit E',
    'Vit K'
]
recipe_df = pd.DataFrame(columns=["Product"]+nutrients)
# Iterate over the ingredients in recipe_data
for ix, ing in enumerate(recipe_data["ingredients"]):
    # print(ing["ingredient"])
    product_data = prod_data.loc[prod_data["Product"] == ing["ingredient"]]

    if not product_data.empty:

        # Extract product info
        nutrient_list = product_data[nutrients].values[0]
        serving_weight = product_data["ServingWeight"].values[0]
        serving_unit = product_data["ServingUnit"].values[0]
        serving_qty = product_data["ServingQty"].values[0]

        # Get product grams
        if ing["amount_unit"] == serving_unit:
            product_grams = ing["amount"] * serving_weight / serving_qty

        elif ing["amount_unit"] == "GRAMS":
            product_grams = ing["amount"]

        else: 
            print(f"Ingredient {ing['ingredient']} does not have correct serving unit.")
            continue

        # Scale nutrient values for product grams
        scaled_nutrients = [(x * product_grams) / serving_weight for x in nutrient_list]

        # Update recipe_df with the scaled nutrients
        recipe_df.at[ix, "Product"] = ing["ingredient"]
        for nutrient, value in zip(nutrients, scaled_nutrients):
            recipe_df.at[ix, nutrient] = value
    else:
        print(f"Ingredient {ing['ingredient']} not found in product data.")

display(recipe_df)

,Product,Calories,Protein,Fiber,TotalCarboHydrates,TotalFat,SaturatedFat,TransFat,PolyUnsaturatedFat,MonoUnsaturatedFat,...,Potassium,Sodium,Zinc,Vit A,Vit B6,Vit B12,Vit C,Vit D,Vit E,Vit K
0,Ramen Noodles,189.700396,4.877049,1.513567,25.948276,7.211702,3.318966,0.0,0.866245,2.489323,...,37.166478,807.888638,0.430921,0.0,0.041203,0.0,0.0,0.0,1.075424,0.514415
1,Bok Choy,122.4,15.9,10.2,18.18,1.62,0.24,0.0,0.7854,0.1224,...,3784.2,346.8,1.734,2162.4,1.6932,0.0,265.2,0.0,0.918,346.8
2,Ginger,2.4,0.06,0.06,0.54,0.03,0.0,0.0,0.00465,0.00465,...,12.45,0.39,0.0102,0.0,0.0048,0.0,0.15,0.0,0.0078,0.003
3,Sesame Oil,120.22,0.0,0.0,0.0,13.6,1.93,NaN,5.6712,5.3992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1904,1.8496
4,Garlic Clove,8.94,0.38,0.12,1.98,0.04,0.0,0.0,0.015,0.0006,...,24.06,1.02,0.0696,0.0,0.0742,0.0,1.872,0.0,0.0048,0.102
5,Onion,41.36,1.28,1.32,9.54,0.18,0.03,0.0,0.0686,0.0254,...,156.04,2.82,0.1974,0.0,0.1213,0.0,4.888,0.0,0.0188,0.47
6,Green Onion,9.6,0.54,0.78,2.2,0.06,0.0,0.0,0.0222,0.0082,...,82.8,4.8,0.117,15.0,0.0184,0.0,5.64,0.0,0.165,62.1
7,Miso Paste,50.49,3.255,1.38,6.465,1.53,0.255,0.0,0.73545,0.28515,...,53.55,950.64,0.6528,1.02,0.0507,0.0204,0.0,0.0,0.00255,7.4715
8,Soy Sauce,12.72,1.95,0.195,1.185,0.135,0.015,0.0,0.06315,0.02115,...,104.4,1318.32,0.2088,0.0,0.03555,0.0,0.0,0.0,0.0,0.0
9,Coconut Milk,925.9,9.503982,NaN,13.205752,100.259735,88.904867,NaN,1.095142,4.262858,...,1034.0,61.1,2.632,0.0,0.131642,0.0,4.7,0.0,NaN,NaN


### Total recipe amounts

In [4]:
total_amounts = recipe_df.drop("Product", axis=1).sum() / recipe_servings
display(total_amounts)

Calories              1102.387698
Protein                 34.784425
Fiber                   25.529283
TotalCarboHydrates      87.773105
TotalFat                79.279809
SaturatedFat            49.226325
TransFat                      0.0
PolyUnsaturatedFat       6.129608
MonoUnsaturatedFat      16.269151
AddedSugars              24.64904
Cholesterol                   0.0
Calcium               3073.112117
Iron                     17.95949
Magnesium             1168.133499
Potassium             4788.725739
Sodium                6891.654319
Zinc                    18.460561
Vit A                  1093.41325
Vit B6                   3.108292
Vit B12                    0.0102
Vit C                    142.3455
Vit D                       2.736
Vit E                    3.306662
Vit K                  219.179532
dtype: object

In [5]:
recipe_df.sort_values("Sodium")

,Product,Calories,Protein,Fiber,TotalCarboHydrates,TotalFat,SaturatedFat,TransFat,PolyUnsaturatedFat,MonoUnsaturatedFat,...,Potassium,Sodium,Zinc,Vit A,Vit B6,Vit B12,Vit C,Vit D,Vit E,Vit K
3,Sesame Oil,120.22,0.0,0.0,0.0,13.6,1.93,NaN,5.6712,5.3992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1904,1.8496
11,Black Pepper,2.885,0.12,0.29,0.735,0.035,0.015,0.0,0.0115,0.0085,...,15.285,0.23,0.0137,0.3105,0.00335,0.0,0.0,0.0,0.01195,1.88255
15,Anise Star,7.08,0.37,0.31,1.05,0.33,0.01,NaN,0.0662,0.2054,...,30.26,0.34,0.1113,0.336,0.0137,0.0,0.441,0.0,NaN,NaN
2,Ginger,2.4,0.06,0.06,0.54,0.03,0.0,0.0,0.00465,0.00465,...,12.45,0.39,0.0102,0.0,0.0048,0.0,0.15,0.0,0.0078,0.003
13,Olive Oil,238.68,0.0,0.0,0.0,27.0,3.72,NaN,2.8412,19.6994,...,0.28,0.54,0.0,0.0,0.0,0.0,0.0,0.0,3.8746,16.254
4,Garlic Clove,8.94,0.38,0.12,1.98,0.04,0.0,0.0,0.015,0.0006,...,24.06,1.02,0.0696,0.0,0.0742,0.0,1.872,0.0,0.0048,0.102
5,Onion,41.36,1.28,1.32,9.54,0.18,0.03,0.0,0.0686,0.0254,...,156.04,2.82,0.1974,0.0,0.1213,0.0,4.888,0.0,0.0188,0.47
14,Tomato Paste,6.56,0.345,0.33,1.515,0.04,0.01,0.0,0.0128,0.00535,...,81.12,4.72,0.0504,6.08,0.0173,0.0,1.752,0.0,0.344,0.912
6,Green Onion,9.6,0.54,0.78,2.2,0.06,0.0,0.0,0.0222,0.0082,...,82.8,4.8,0.117,15.0,0.0184,0.0,5.64,0.0,0.165,62.1
17,Vegetable Broth Cube,0.72,0.027818,0.0,0.122182,0.008182,0.003818,0.0,0.00048,0.00072,...,2.4,36.36,0.0036,1.68,0.00084,NaN,0.048,NaN,0.0,NaN
